# CS285Project
**Student:** Abdulleh Alafari — **ID:** 224111642
- November 2025 - Fall 2025 Semester
## Overview
**Diffie–Hellman (DH)** key exchange, **RSA** encryption/decryption, and **HMAC** (with the DH key) for integrity.

## Project Layout
```text
demo/
├─ Server.py          # Generates RSA, DH, decrypts, verifies HMAC
├─ Client.py          # Connects to server, DH, encrypts (RSA), signs (HMAC)
└─ commonSpace.py     # Shared params (q, a) and a helper (takein)

## Common parameters and helper (demo/commonSpace.py)
This module defines the shared Diffie–Hellman parameters and a small helper:
- **q**: a small prime used for demo purposes
- **a**: a primitive root modulo q
- **takein(connection)**: reads up to 1024 bytes from a socket to simplify message receiving.

These are imported by both `Server.py` and `Client.py`.


In [ ]:
# demo/commonSpace.py
# q and a are agreed on numbers
# q in a prime while and a  is a premitive root of q
q = 7
a = 5

import socket
#to make recieving data easier in each class
def takein(connection : socket.socket):
    
    try:
        while True:
            data = connection.recv(1024)
            if not data:
                break
            return (data) 
    finally:
        pass


## Server (demo/Server.py)
The server:
- **Generates RSA** keypair and shares the public key.
- **Listens** on `localhost:6000` and accepts a client connection.
- **Performs Diffie–Hellman** exchange to derive a shared secret `K`.
- **Receives an RSA-encrypted message** and decrypts it.
- **Verifies HMAC** on subsequent RSA-encrypted messages using `K` for integrity before decrypting.


In [ ]:
# demo/Server.py
import socket
import secrets
# Set up q and a for DH key generation
from commonSpace import takein, a , q
from Crypto.PublicKey import RSA
import hashlib, hmac
    
        
        
# =-=-=-=-=-=-=-=-=-= Generate RSA key Pair =-=-=-=-=-=-=-=-=-=
key = RSA.generate(bits=1024)
private_rsa, public_rsa = key.export_key(), key.public_key().export_key()


# =-=-=-=-=-=-=-=-=-= Set up a socket and connection =-=-=-=-=-=-=-=-=-=
# socket.socket(socket_family, socket_type) -> AF_INET - IPv4, SOCK_STREAM - TCP
serverSocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# socket.bind((IP_Address, PORT)) - Tuple not 1 Argument - binds socket to address
serverSocket.bind(('localhost', 6000))
# socket.listen(number of connections to wait before calling accept)
serverSocket.listen(1)
# Socket Connection, IP Address of client
connection, ipAdd = serverSocket.accept()


# =-=-=-=-=-=-=-=-=-= Share RSA Public Key =-=-=-=-=-=-=-=-=-=
connection.send(public_rsa)



# =-=-=-=-=-=-=-=-=-= Diffie-Hellman Setup =-=-=-=-=-=-=-=-=-=
# q and a are agreed on numbers
# q in a prime while and a  is a premitive root of q
q, a = q, a
Xa = secrets.randbelow(q) # random number in bellow q
Ya = int(pow(a,Xa, q)) # calculate public key
# =-=-=-=-=-=-=-=-=-= Exchange Diffie-Hellman Public keys =-=-=-=-=-=-=-=-=-=
connection.send(str(Ya).encode())
Yb = int(takein(connection))
# =-=-=-=-=-=-=-=-=-= Diffie-Hellman Secret Key =-=-=-=-=-=-=-=-=-=
K = pow(Yb,Xa,q)
# print("Ya = ", Ya, "Yb = ", Yb)
print("Diffie-Hellman key:", K)


# =-=-=-=-=-=-=-=-=-= Recive Encoded Message =-=-=-=-=-=-=-=-=-=
cipherbin = takein(connection)
# =-=-=-=-=-=-=-=-=-= Decode Encoded Message =-=-=-=-=-=-=-=-=-=
cipher_int = int(cipherbin.decode())
decrypt_int = pow(cipher_int,key.d, key.n)
# The formula is to calculate to_bytes arguments 
# (decrypt.bit_length() + 7)//8) to find the number of bits needed to represent decrypt_int
plaintext = decrypt_int.to_bytes((decrypt_int.bit_length() + 7)//8).decode()
print(plaintext)


# =-=-=-=-=-=-=-=-=-= 1st Recive Encoded Message With Ingegrity =-=-=-=-=-=-=-=-=-=
cipherbin, signature = takein(connection).split(b"\n",1)
expectedSignature = hmac.new(str(K).encode(), cipherbin, hashlib.sha256).digest() + int.to_bytes(1)
# =-=-=-=-=-=-=-=-=-= Decode Encoded Message Ingegrity =-=-=-=-=-=-=-=-=-=
if (hmac.compare_digest(signature, expectedSignature)):
    cipher_int = int(cipherbin.decode())
    decrypt_int = pow(cipher_int,key.d, key.n)
    # The formula is to calculate to_bytes arguments 
    # (decrypt.bit_length() + 7)//8) to find the number of bits needed to represent decrypt_int
    plaintext = decrypt_int.to_bytes((decrypt_int.bit_length() + 7)//8).decode()
    print("First Attempt: ", plaintext)
else:
    print("not expected signature")

# =-=-=-=-=-=-=-=-=-= 2nd Recive Encoded Message With Ingegrity =-=-=-=-=-=-=-=-=-=
cipherbin, signature = takein(connection).split(b"\n",1)
expectedSignature = hmac.new(str(K).encode(), cipherbin, hashlib.sha256).digest()
# =-=-=-=-=-=-=-=-=-= Decode Encoded Message Ingegrity =-=-=-=-=-=-=-=-=-=
if (hmac.compare_digest(signature, expectedSignature)):
    cipher_int = int(cipherbin.decode())
    decrypt_int = pow(cipher_int,key.d, key.n)
    # The formula is to calculate to_bytes arguments 
    # (decrypt.bit_length() + 7)//8) to find the number of bits needed to represent decrypt_int
    plaintext = decrypt_int.to_bytes((decrypt_int.bit_length() + 7)//8).decode()
    print("Second Attempt: ", plaintext)
else:
    print("not expected signature")


## Client (demo/Client.py)
The client:
- **Connects** to `localhost:6000`.
- **Receives** the server's RSA public key.
- **Performs Diffie–Hellman** to derive the same shared secret `K`.
- **Encrypts messages with RSA** using the server's public key and sends them.
- **Computes HMAC** (with `K`) over the ciphertext to provide message integrity.


In [ ]:
# demo/Client.py
from pydoc import plain
import socket
import secrets
import hmac, hashlib
from time import sleep
from commonSpace import takein, a, q
from Crypto.PublicKey import RSA
# =-=-=-=-=-=-=-=-=-= Set up a socket and connect as client =-=-=-=-=-=-=-=-=-=
client = socket.socket()
client.connect(('localhost', 6000))



# =-=-=-=-=-=-=-=-=-= Recieve RSA Public Key of Server =-=-=-=-=-=-=-=-=-=
public_rsa = RSA.import_key(takein(client)) # convert from bytes to key

# =-=-=-=-=-=-=-=-=-= Diffie-Hellman Setup =-=-=-=-=-=-=-=-=-=
# q and a are agreed on numbers
# q in a prime while and a  is a premitive root of q
q, a = q, a
Xb = secrets.randbelow(q) # random number bellow q
Yb = int(pow(a,Xb, q)) # calculate public key
# =-=-=-=-=-=-=-=-=-= Exchange Diffie-Hellman Public keys =-=-=-=-=-=-=-=-=-=
client.send(str(Yb).encode())
Ya = int(takein(client))
# =-=-=-=-=-=-=-=-=-= Diffie-Hellman Secret Key =-=-=-=-=-=-=-=-=-=
K = pow(Ya,Xb,q)
# print("Ya = ", Ya, "Yb = ", Yb)
print(K)
print("Diffie-Hellman key:", K)


# =-=-=-=-=-=-=-=-=-= Send Encoded Message =-=-=-=-=-=-=-=-=-=
plain_message = "Hello This is me Client"
cipherint = pow(int.from_bytes(plain_message.encode()),public_rsa.e, public_rsa.n) 
client.send(str(cipherint).encode())
print(plain_message)

# =-=-=-=-=-=-=-=-=-= Send Encoded Message With Ingegrity =-=-=-=-=-=-=-=-=-=
# Will use K from Diffle Hellman to sign the message
plain_message = "Hello This is me Client. Please check Integrity"
cipherint = pow(int.from_bytes(plain_message.encode()),public_rsa.e, public_rsa.n) 
cipher_byte = str(cipherint).encode()
# str(K).encode() to convert K to bytes
# 
signature = hmac.new(str(K).encode(), cipher_byte, hashlib.sha256).digest()
client.sendall(cipher_byte +b"\n"+ signature)

sleep(10)
# =-=-=-=-=-=-=-=-=-= Send Encoded Message With Ingegrity =-=-=-=-=-=-=-=-=-=
# Will use K from Diffle Hellman to sign the message
plain_message = "Hello This is me Client. Please check Integrity"
cipherint = pow(int.from_bytes(plain_message.encode()),public_rsa.e, public_rsa.n) 
cipher_byte = str(cipherint).encode()
# str(K).encode() to convert K to bytes
# 
signature = hmac.new(str(K).encode(), cipher_byte, hashlib.sha256).digest()
client.sendall(cipher_byte +b"\n"+ signature)


## How to run (for reference)
- Open two terminals in the `demo/` directory.
- Terminal 1: run the server `python3 Server.py`.
- Terminal 2: run the client `python3 Client.py`.

Notes:
- The code in this notebook mirrors `demo/*.py`.
- Requires `pycryptodome` (`pip install pycryptodome`).
